# Lab 2

breast-cancer.arff

*C4.5 decicion tree* vs *naive bayes classification*

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.io import arff
import pandas as pd
import numpy as np
import matplotlib
from sklearn import tree
import graphviz

In [2]:
data = arff.loadarff('breast-cancer.arff')
df = pd.DataFrame(data[0]).select_dtypes([np.object]).stack().str.decode('utf-8').unstack()
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,40-49,premeno,15-19,0-2,yes,3,right,left_up,no,recurrence-events
1,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
2,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
3,40-49,premeno,35-39,0-2,yes,3,right,left_low,yes,no-recurrence-events
4,40-49,premeno,30-34,3-5,yes,2,left,right_up,no,recurrence-events


In [3]:
df.iloc[20:21,] # row with missing data

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
20,50-59,lt40,20-24,0-2,?,1,left,left_low,no,recurrence-events


In [4]:
df.index.size

286

In [5]:
df = df[(df['node-caps'] != '?') & (df['breast-quad'] != '?')]

In [6]:
df.index.size

277

In [7]:
df.shape

(277, 10)

In [15]:
cats = {}
codes, cats['age'] = pd.factorize(df['age'])
df.loc[:,'age'] = codes
codes, cats['menopause'] = pd.factorize(df['menopause'])
df.loc[:,'menopause'] = codes
codes, cats['tumor-size'] = pd.factorize(df['tumor-size'])
df.loc[:,'tumor-size'] = codes
codes, cats['inv-nodes'] = pd.factorize(df['inv-nodes'])
df.loc[:,'inv-nodes'] = codes
codes, cats['node-caps'] = pd.factorize(df['node-caps'])
df.loc[:,'node-caps'] = codes
codes, cats['breast'] = pd.factorize(df['breast'])
df.loc[:,'breast'] = codes
codes, cats['breast-quad'] = pd.factorize(df['breast-quad'])
df.loc[:,'breast-quad'] = codes
codes, cats['irradiat'] = pd.factorize(df['irradiat'])
df.loc[:,'irradiat'] = codes
codes, cats['Class'] = pd.factorize(df['Class'])
df.loc[:,'Class'] = codes
df.loc[:,'deg-malig'] = df['deg-malig'].astype(int)


In [16]:
corrMat = df.corr()

corrMat.style.background_gradient(cmap='coolwarm')

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
age,1.000000,0.242579,-0.031988,-0.029323,0.036850,-0.019273,0.073121,-0.018384,-0.029499,-0.023966
menopause,0.242579,1.000000,0.006746,-0.035095,0.048569,0.040020,0.059217,-0.092046,-0.093481,0.087480
tumor-size,-0.031988,0.006746,1.000000,-0.105610,0.078192,-0.103100,-0.020722,-0.047194,0.005344,0.079440
inv-nodes,-0.029323,-0.035095,-0.105610,1.000000,-0.632495,0.333198,0.021855,0.080266,0.370313,-0.317881
node-caps,0.036850,0.048569,0.078192,-0.632495,1.000000,-0.330181,-0.012345,-0.043770,-0.333537,0.289002
deg-malig,-0.019273,0.040020,-0.103100,0.333198,-0.330181,1.000000,0.045874,-0.040290,0.230821,-0.330253
breast,0.073121,0.059217,-0.020722,0.021855,-0.012345,0.045874,1.000000,0.168374,-0.025230,-0.041304
breast-quad,-0.018384,-0.092046,-0.047194,0.080266,-0.043770,-0.040290,0.168374,1.000000,-0.006060,-0.055458
irradiat,-0.029499,-0.093481,0.005344,0.370313,-0.333537,0.230821,-0.025230,-0.006060,1.000000,-0.226020
Class,-0.023966,0.087480,0.079440,-0.317881,0.289002,-0.330253,-0.041304,-0.055458,-0.226020,1.000000


In [10]:
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,0,0,0,0,0,3,0,0,0,0
1,1,1,0,0,1,1,0,1,0,1
2,1,1,1,0,1,2,1,2,0,0
3,0,0,1,0,0,3,0,2,1,1
4,0,0,2,1,0,2,1,3,0,0


PErmutation and splitting of the dataset

In [191]:
Y = df['Class']
X = df.iloc[:,:-1]

m = df.shape[0]
print(m)
permutation = np.random.permutation(m) # random permurtation

X = X.iloc[permutation]
Y = Y.iloc[permutation]

m_training=round(m*80/100)

#patterns in the test set
m_test=m-m_training

#X_training = instances for training set
X_training =X[0:m_training] 

#Y_training = labels for the training set
Y_training = Y[0:m_training]

#X_test = instances for test set
X_test = X[m_training-1:-1]

#Y_test = labels for the test set
Y_test = Y[m_training-1:-1]


277


In [322]:

#without features reduction
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_training, Y_training)
acc = clf.score( X_test, Y_test)

print(acc)


0.7272727272727273


In [105]:
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("decisiontree")

'decisiontree.pdf'

In [354]:
#without age
X_training_age= X_training[X_training.columns.difference(['age'])]
X_test_age= X_test[X_test.columns.difference(['age'])]

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_training_age, Y_training)
acc = clf.score( X_test_age, Y_test)

print(acc)

0.6363636363636364


In [428]:
#without breast
X_training_breast= X_training[X_training.columns.difference(['breast'])]
X_test_breast= X_test[X_test.columns.difference(['age'])]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_training_breast, Y_training)
acc = clf.score( X_test_breast, Y_test)

print(acc)

0.6363636363636364


In [108]:
#without age & breast
X_training_age_breast= X_training[X_training.columns.difference(['age','breast'])]
X_test_age_breast= X_test[X_test.columns.difference(['age','breast'])]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_training_age_breast, Y_training)
acc = clf.score( X_test_age_breast, Y_test)

print(acc)

0.5636363636363636
